# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vadso,70.0744,29.7487,-1.91,59,100,8.23,NO,1712530039
1,1,port mathurin,-19.6833,63.4167,26.86,79,18,4.66,MU,1712530039
2,2,puerto natales,-51.7236,-72.4875,3.99,92,100,5.45,CL,1712529900
3,3,puerto deseado,-47.7503,-65.8938,11.98,54,1,7.39,AR,1712530040
4,4,bam,29.1060,58.3570,23.02,16,62,4.30,IR,1712530041


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [31]:
%%capture --no-display

# Configure the map plot
plot_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 500,
    color="City",
    alpha=0.5,
    s="Humidity",
    scale=0.8)

# Display the map
plot_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [24]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df['Max Temp']<=22)
                                    &(city_data_df['Max Temp']>=0)
                                    &(city_data_df['Cloudiness']<=80)
                                    &(city_data_df['Humidity']<80)
                                    &(city_data_df['Country']!='US')
                                    &(city_data_df['Country']!='CN')
                                    &(city_data_df['Country']!='UK'), :]
print(len(ideal_weather_df))

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

74


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,puerto deseado,-47.7503,-65.8938,11.98,54,1,7.39,AR,1712530040
6,6,geraldton,-28.7667,114.6000,19.23,63,77,1.03,AU,1712530041
26,26,krasnogvardeyskoye,45.8456,41.5186,10.71,61,10,3.97,RU,1712530049
38,38,smara,26.7384,-11.6719,16.64,75,40,7.01,EH,1712530053
39,39,yellowknife,62.4560,-114.3525,4.29,70,75,5.66,CA,1712529880


### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
3,puerto deseado,AR,-47.7503,-65.8938,54,
6,geraldton,AU,-28.7667,114.6000,63,
26,krasnogvardeyskoye,RU,45.8456,41.5186,61,
38,smara,EH,26.7384,-11.6719,75,
39,yellowknife,CA,62.4560,-114.3525,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lon = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
puerto deseado - nearest hotel: Los Olmos
geraldton - nearest hotel: Ocean Centre Hotel
krasnogvardeyskoye - nearest hotel: No hotel found
smara - nearest hotel: Hôtel Amine فندق الأمين
yellowknife - nearest hotel: No hotel found
benghazi - nearest hotel: ترجاب
astrakhan - nearest hotel: Omega
la ligua - nearest hotel: Hotel Anchimallen
yelizovo - nearest hotel: Николина Усадьба
at taj - nearest hotel: No hotel found
la ronge - nearest hotel: La Ronge Hotel and Suites
kropyvnytskyi - nearest hotel: Шенген
whitehorse - nearest hotel: Town & Mountain Hotel
thompson - nearest hotel: Thompson Inn
stanley - nearest hotel: Hotel 52
maloye ganyushkino - nearest hotel: Конил Сазы
guerrero negro - nearest hotel: Plaza sal paraiso
margaret river - nearest hotel: Margaret River Hotel
saint-francois - nearest hotel: Chez Lily
san juanito - nearest hotel: Cabañas Presa Situriachi
ak-suu - nearest hotel: BellavistaHotel
al kharijah - nearest hotel: Kharga Hotel
selfoss - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
3,puerto deseado,AR,-47.7503,-65.8938,54,Los Olmos
6,geraldton,AU,-28.7667,114.6000,63,Ocean Centre Hotel
26,krasnogvardeyskoye,RU,45.8456,41.5186,61,No hotel found
38,smara,EH,26.7384,-11.6719,75,Hôtel Amine فندق الأمين
39,yellowknife,CA,62.4560,-114.3525,70,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot
plot_hotel = hotel_df.hvplot.points("Lng",
                "Lat",
                geo=True,
                frame_width=500,
                frame_height=500,
                color="City",
                alpha=0.5,
                hover=True,
                hover_cols="all",
                use_index=False,
                s="Humidity",
                scale=1)

# Display the map
plot_hotel

:Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)